<a href="https://colab.research.google.com/github/andreidm92/computer_vision_tasks/blob/main/practice/Lesson_20_crowd_ant_counting_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐜🧍 Проект: Подсчёт муравьёв и людей в толпе с использованием трёх методов


## 🎯 Цель проекта
Сравнить три современных подхода к подсчёту объектов:
1. **CSRNet** — подсчёт по картам плотности (density map regression)
2. **Mask R-CNN** — instance segmentation (пиксельные маски)
3. **YOLACT** — real-time instance segmentation

Примеры:
- 🧍 Толпа людей — [NWPU-Crowd](https://gjy3035.github.io/NWPU-Crowd-Sample-Code/)
- 🐜 Муравьи — [Ant Detection Dataset (Roboflow)](https://universe.roboflow.com/datasets-rxtdh/ant-detection-gfju4-ooiag)

В проект входят:
- Теория (архитектуры, формулы, компоненты)
- Практика на датасетах
- OpenCV-решение
- Линейная алгебра (SVD, псевдообратная матрица)



### 🧱 VGG16
- Глубокая CNN, состоящая из **13 сверточных слоёв** и **3 полносвязных**
- Используется как feature extractor в CSRNet
- Архитектура: последовательные Conv3x3 + ReLU + MaxPool

**В CSRNet**: используется **только сверточная часть** (без FC)
```plaintext
Conv3-64 → Conv3-64 → MP →
Conv3-128 → Conv3-128 → MP →
Conv3-256 ×3 → MP →
Conv3-512 ×3 → MP →
Conv3-512 ×3
```

---

### 🔁 Dilated Convolutions
- Расширенные свёртки: увеличивают receptive field **без увеличения числа параметров**
- Используются в CSRNet **вместо Pooling** после VGG16
- Позволяют видеть больше контекста в изображении

---

### 🌀 ResNet
- Основная сеть в Mask R-CNN и YOLACT
- Идея: **residual connection** — $F(x) + x$
- Примеры: ResNet-50 (50 слоёв), ResNet-101
- Состоят из Bottleneck-блоков

---

### 🧭 FPN (Feature Pyramid Network)
- Усиливает ResNet: объединяет признаки с разных уровней (низкий, средний, высокий)
- Полезно при обнаружении объектов **разного размера**
- Используется и в Mask R-CNN, и в YOLACT

---

### 🧿 RPN (Region Proposal Network)
- Генерирует **anchor**-области, где могут находиться объекты
- Выход: bbox и objectness score
- Работает поверх FPN

---

### 🎯 RoIAlign
- Операция выбора фичей из предложенных RPN region
- В отличие от RoIPool **не округляет координаты**
- Обеспечивает точную маску

---

### 🎨 Protonet (в YOLACT)
- Маленькая CNN, генерирует **N прототипов масок** для изображения
- Вместо одной маски для каждого объекта → комбинация прототипов

```plaintext
M_объекта = sum(c_i × P_i), i = 1..N
```


## ⚙️ Подготовка среды

In [ ]:

# Базовые библиотеки
!pip install -q torch torchvision opencv-python matplotlib

# Detectron2 (Mask R-CNN)
!pip install -q git+https://github.com/facebookresearch/detectron2.git


## 🧪 Задание 1: CSRNet на NWPU и муравьях

In [ ]:

# Пример генерации density map из dot-аннотаций (heatmap)
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

def generate_density_map(image_shape, points, sigma=4):
    density = np.zeros(image_shape[:2], dtype=np.float32)
    for x, y in points:
        if x >= image_shape[1] or y >= image_shape[0]:
            continue
        density[int(y), int(x)] += 1
    return gaussian_filter(density, sigma=sigma)

# Пример: 3 точки
img_shape = (256, 256, 3)
dots = [(100, 100), (120, 140), (200, 220)]
density_map = generate_density_map(img_shape, dots)

plt.imshow(density_map, cmap='jet')
plt.title("Density Map")
plt.colorbar()
plt.show()


## 📘 Теория: модели и архитектуры

## 🧠 Метод 1: CSRNet — Density Map Regression


CSRNet (Congested Scene Recognition Network) — это сверточная нейронная сеть, предназначенная для **подсчёта объектов** в **плотных сценах**, таких как толпы людей, муравьи, стаи птиц и т.д.

### 🧩 Архитектура:
CSRNet состоит из двух частей:
1. **Backbone** — извлекает признаки: используется **VGG16** без полносвязных слоёв
2. **Регрессионный блок** — использует **Dilated Convolutions** для увеличения поля зрения

### 🔧 Зачем нужны компоненты:

- **VGG16** (13 conv-слоёв):
  Извлекает признаки из изображения. Слои:
  ```
  Conv3-64 → Conv3-64 → MP →
  Conv3-128 → Conv3-128 → MP →
  Conv3-256 ×3 → MP →
  Conv3-512 ×3 → MP →
  Conv3-512 ×3
  ```

- **Dilated Convolution Layers**:
  Увеличивают область захвата (receptive field), не увеличивая параметров. Используются вместо обычных свёрток:
  ```
  Conv(3x3, dilation=2), ReLU → повторяется несколько раз
  ```

Эти слои позволяют сети «видеть» объекты в широком контексте и адекватно строить карту плотности.

### 📐 Что предсказывает CSRNet?
2D-изображение (density map), где яркость каждого пикселя = «плотность объектов» в этом месте. Сумма всех значений ≈ количество объектов.



## 🧠 Метод 2: Mask R-CNN — Instance Segmentation


Mask R-CNN — это модель для **instance segmentation**, то есть предсказания **bounding box + маски** для каждого объекта на изображении.

### 🧩 Архитектура:
Mask R-CNN состоит из:
1. **Backbone (ResNet-50 или 101)** + **FPN**
2. **RPN (Region Proposal Network)** — генерирует регионы
3. **RoIAlign** — извлекает фичи из регионов
4. **Heads** — классификация, bbox-регрессия, маска

### 🔧 Компоненты:

- **ResNet-50/101**:
  Извлекает признаки. Используются блоки типа bottleneck:
  ```
  Conv1 → MaxPool →
  Conv2_x (3 блока) →
  Conv3_x (4 блока) →
  Conv4_x (6 или 23 блока) →
  Conv5_x (3 блока)
  ```

- **FPN (Feature Pyramid Network)**:
  Объединяет признаки с разных уровней, чтобы находить объекты **разного масштаба**.

- **RPN (Region Proposal Network)**:
  Генерирует «якоря» — регионы, где, вероятно, есть объект.

- **RoIAlign**:
  Извлекает фиксированный размер признаков из предложенных регионов, **без округлений координат**.

- **Mask Head**:
  Маленькая CNN, предсказывает маску 28×28 на каждый найденный объект.



## 🧠 Метод 3: YOLACT — Real-Time Instance Segmentation


YOLACT (You Only Look At Coefficients) — это модель для instance segmentation, построенная для **реального времени**.

### 🧩 Архитектура:
1. **Backbone (ResNet-50/101)** + **FPN**
2. **Protonet** — генерирует прототипы масок
3. **Prediction head** — bbox, классы, маска-коэффициенты

### 🔧 Компоненты:

- **ResNet** + **FPN**:
  Извлекает признаки, работает как в Mask R-CNN.

- **Protonet**:
  Маленькая CNN (~3–5 conv-слоёв), создаёт **N прототипов** (например, 32 маски на всё изображение).

- **Коэффициенты маски**:
  Для каждого объекта считаются весы $c_i$, итоговая маска:
  $$
  M = \sum_{i=1}^{N} c_i \cdot P_i
  $$

  где $P_i$ — прототип, $c_i$ — коэффициент.

Такой подход позволяет ускорить сегментацию, так как прототипы общие, и не надо строить маску с нуля для каждого объекта.



## 🧪 Практика: загрузка данных, обучение моделей, предсказание

## 🚀 Обучение CSRNet


CSRNet обычно требует dot-аннотаций (формат `.mat` или `.txt` с координатами объектов). Мы используем `ShanghaiTech`, `NWPU` или кастомный набор.

### 📦 Шаги:
1. Сформировать ground truth density maps
2. Обучить CSRNet на них (используется PyTorch)
3. Валидировать на отдельной выборке

📌 Подробный код обучения см. в [оригинальном репозитории CSRNet](https://github.com/leeyeehoo/CSRNet-pytorch)

### 🔧 Типичный фрагмент кода:
```python
from model import CSRNet
from torch.utils.data import DataLoader
from dataset import CrowdDataset

model = CSRNet().cuda()
train_loader = DataLoader(...)

for batch in train_loader:
    images, gt_density = batch
    output = model(images.cuda())
    loss = loss_fn(output, gt_density.cuda())
    ...
```


## 🚀 Обучение Mask R-CNN (Detectron2)

In [ ]:

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # муравьи

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


## 🚀 Обучение YOLACT


YOLACT нужно устанавливать из исходников:  
🔗 [https://github.com/dbolya/yolact](https://github.com/dbolya/yolact)

### 🔧 Установка:
```bash
!git clone https://github.com/dbolya/yolact.git
%cd yolact
!pip install -r requirements.txt
!python setup.py build_ext --inplace
```

### 📦 Обучение:
1. Подготовить датасет в формате COCO
2. Изменить конфигурацию в `config.py`
3. Запуск:
```bash
python train.py --config=yolact_base_config --batch_size=8
```


## 🧪 Задание 2: решение задачи средствами OpenCV

## 🧪 Задание 2: OpenCV подход к подсчёту объектов (через connected components)

In [ ]:

# Простая бинаризация + connected components
thresh = cv2.threshold((density_map * 255).astype(np.uint8), 10, 255, cv2.THRESH_BINARY)[1]
num_labels, labels_im = cv2.connectedComponents(thresh)

print("Объектов найдено (по компонентам):", num_labels - 1)
plt.imshow(thresh, cmap='gray')
plt.title("Binary Thresholded Map")
plt.show()


## 🧮 Задание 3: линейная алгебра — SVD и псевдообратная матрица

## 🧮 Задание 3: Линейная алгебра — SVD и псевдообратная матрица

In [ ]:

import numpy as np

A = np.array([[1, 2], [3, 4], [5, 6]])
U, S, VT = np.linalg.svd(A)
print("U:
", U)
print("S:
", S)
print("VT:
", VT)

# Построим псевдообратную
Sigma_plus = np.zeros((VT.shape[0], U.shape[0]))
Sigma_plus[:len(S), :len(S)] = np.diag(1/S)

A_pseudo = VT.T @ Sigma_plus @ U.T
print("Псевдообратная A⁺:
", A_pseudo)

# Решим Ax = b
b = np.array([1, 2, 3])
x = A_pseudo @ b
print("x (решение):", x)
